In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

In [4]:
LAG_SIZE = 61.4        # Размер лага
COUNT_OF_LAGS = 15     # Количество лагов 
D_ANGLE = 25           # Допуск угла
WIDTH_OF_STRIPE = 1000 # Ширина бина (полосы) 
MAX_DIST = 10000000

In [5]:
def caclulate_length_by_two_points(point_1, point_2):
    """Вычисляет расстояние по двум точкам"""
    x1 = point_1[0]
    y1 = point_1[1]
    
    x2 = point_2[0]
    y2 = point_2[1]
    
    length = np.sqrt(((x1 - x2) ** 2) + (y1 - y2) ** 2)
    return length

In [6]:
def inverse_geodesic_task(p1, p2):
    """Обратная геодезическая задача, по двум точкам определяет дирекционный угол"""
    x1 = p1[0]
    y1 = p1[1]
    
    x2 = p2[0]
    y2 = p2[1]
    
    dy = y2 - y1
    dx = x2 - x1
    if dx == 0:
        dx = 0.0000000000000000000001
    rumb = abs(np.rad2deg(np.arctan(dy / dx)))
    
    if dy >= 0 and dx >= 0:
        direction_angel = rumb
    elif dy >= 0 and dx <= 0:
        direction_angel = 180 - rumb
    elif dy <= 0 and dx <= 0:
        direction_angel = 180 + rumb
    else:
        direction_angel = 360 - rumb
    return direction_angel

In [7]:
def to_dir_angle(angle):
    """ Конвертирует угол в дирекционный угол """
    if angle > 360:
        return 360 - angle
    elif angle < 0:
        return angle + 360
    else:
        return angle

In [8]:
def direct_geodesic_problem(p1, dist, angle):
    """Прямая геодизмческая задача. По точке, расстоянию и дирекционному углу вычисляет \
        координаты второй точки"""
    dx = np.cos(np.deg2rad(angle)) * dist
    dy = np.sin(np.deg2rad(angle)) * dist
    return p1[0] + dx, p1[1] + dy

In [9]:
def calculate_dist_to_line(p1, p2, p3):
    """Расстояние от точки p3 до прямой, заданной двумя точками (p1 и p2) """
    return abs((p2[1] - p1[1]) * p3[0] - (p2[0] - p1[0]) * p3[1] + p2[0] * p1[1] - p2[1] * p1[0]) / MAX_DIST

In [10]:
def caluclate_one_bin(data, indx_of_point, angle):
    """
    Рассчитывает одну полосу (бин) с заданным дирекционным углом (angle) в точке (indx_of_point)
    Возвращает бин разбитый на лаги, в которых содержится кортеж (расстояние до точки, индекс точки в массиве data)
    """

    bin = [[] for i in range(COUNT_OF_LAGS)]

    p1 = (data.iloc[indx_of_point]['X'], data.iloc[indx_of_point]['Y'])
    left_angle = to_dir_angle(angle - D_ANGLE)
    right_angle = to_dir_angle(angle + D_ANGLE)

    p2 = direct_geodesic_problem(p1, MAX_DIST, angle)

    max_lenght_of_bin = COUNT_OF_LAGS * LAG_SIZE

    for new_point_idx in data.index:
        if new_point_idx != indx_of_point:
            p_new = (data.iloc[new_point_idx]['X'], data.iloc[new_point_idx]['Y'])
            dir_angle = inverse_geodesic_task(p1, p_new)
            dist = caclulate_length_by_two_points(p1, p_new)

            if dist > max_lenght_of_bin:
                continue

            if dir_angle > left_angle and dir_angle < right_angle:
                if calculate_dist_to_line(p1, p2, p_new) <= WIDTH_OF_STRIPE / 2:
                    bin[int(dist / LAG_SIZE)].append((dist, new_point_idx))
    
    return bin



In [37]:
def calculate_variogram(data, angle, col):
    """
     Вычисляет вариограмму по заданному дирекционному углу (angle) и целевой переменной (col) 
     Возврат: список средних расстояний в лагах, список значений вариограммы в лагах, список количества точек в лагах  
    """
    all_lags = [[] for i in range(COUNT_OF_LAGS)]

    for idx in data.index:
        bin = caluclate_one_bin(data, idx, angle, col)
        for lag_idx in range(len(bin)):
            if len(bin[lag_idx]) > 0:
                for point in bin[lag_idx]:
                    all_lags[lag_idx].append([point, idx])

    result_distances = []
    result_values = []
    result_counts = []
    for lag in all_lags:
        if len(lag) > 0:
            sum_dist = 0
            sum_std_z = 0
            for item in lag:
                sum_dist += item[0][0]
                sum_std_z += (data.iloc[item[0][1]][col] - data.iloc[item[1]][col]) ** 2
            result_distances.append(sum_dist / len(lag))
            result_values.append(sum_std_z / (2 * len(lag)))
            result_counts.append(len(lag))
        else:
            result_distances.append(None)
            result_values.append(None)
            result_counts.append(0)
    return result_distances, result_values, result_counts


In [40]:
# Направление для построения вариограммы
dir_angle_diraction = 45 

# ШАПКА СЛОБЦА В КОТОРОМ СОДЕРЖИТСЯ ЦЕЛЕВАЯ ПЕРЕМЕННАЯ
column = 'M'

data = pd.read_excel('lab2_data.xlsx')
dists, values, counts = calculate_variogram(data, dir_angle_diraction, column)

my_dict = {'dist': [], 'value': [], 'count': []}
for i in range(len(dists)):
    my_dict['dist'].append(dists[i])
    my_dict['value'].append(values[i])
    my_dict['count'].append(counts[i])
result_df = pd.DataFrame(my_dict)
result_df.to_excel(f'result_{column}.xlsx')

In [41]:
fig = px.line(result_df, x='dist', y="value", text="count")
fig.update_traces(textposition="top right")
fig.show()